In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load DenseNet121
model = models.densenet121(pretrained=True)

# Modify the classifier for your dataset
num_ftrs = model.classifier.in_features
num_classes = 52  # Change this based on your dataset
model.classifier = nn.Linear(num_ftrs, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [6]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root="dataset/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root="dataset/val", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    # Validation 
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)  # Compute validation loss

            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)  # Get predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.2f}%\n")      

Epoch [1/10]
Train Loss: 3.4393 | Val Loss: 12.5477 | Val Acc: 1.92%

Epoch [2/10]
Train Loss: 2.3463 | Val Loss: 3.0879 | Val Acc: 14.81%

Epoch [3/10]
Train Loss: 1.5990 | Val Loss: 11.1502 | Val Acc: 4.62%

Epoch [4/10]
Train Loss: 0.9786 | Val Loss: 4.5466 | Val Acc: 10.00%

Epoch [5/10]
Train Loss: 0.5556 | Val Loss: 5.6403 | Val Acc: 9.81%



KeyboardInterrupt: 